# 2016 총선 지역구 개표결과

: [Computational Documents](https://jhgan00.github.io/computational_docs/widgets_cross.html)에 사용된 데이터의 전처리 과정

## 1. 선거 데이터 수합

공공데이터포털에서 [2016년 국회의원 총선거 개표 데이터](https://www.data.go.kr/dataset/15025527/fileData.do)를 다운받은 후 서울 지역구 결과만 정리한다. 선거구별 데이터를 읽어와 하나로 합쳐주고, 필요한 전처리를 진행해준다. 총 49개 선거구 데이터가 모두 병합되었다.

In [1]:
import pandas as pd
import os
import re

In [2]:
datalist = []
for file in os.listdir("data/seoul"):
    data = pd.read_excel(f"data/seoul/{file}")
    선거구 = file.split("_")[-1].split(".xlsx")[0]
    자치구 = data.iloc[1,0].split("][")[-1][:-1]
    tmp = data[data['Unnamed: 1'] == "소계"].iloc[:,:7]
    tmp.columns = ['읍면동명','소계','선거인수','투표수','새누리당','더불어민주당','국민의당']
    tmp = tmp.assign(자치구= 자치구, 선거구=선거구)
    datalist.append(tmp)

In [3]:
data = pd.concat(datalist).drop("소계",axis=1)

In [4]:
# 총 49개 선거구 데이터
data.선거구.unique().size

49

In [5]:
data = data.assign(읍면동명 = data.읍면동명.apply(lambda x: x.replace("제","",1) if re.search("\d",x) else x))
data = data.assign(읍면동명 = data.읍면동명.apply(lambda x: x.replace("·",".")))

## 2. 행정동코드 매칭

행정동 코드 파일을 읽어와 개표 데이터에 코드를 매칭해준다.

In [6]:
codes = pd.read_csv("data/code.csv")
codes = codes[codes.adm_nm.apply(lambda x: "서울특별시" in x)]
codes = codes.assign(자치구 = codes.adm_nm.apply(lambda x: x.split(" ")[-2]),
                    읍면동명 = codes.adm_nm.apply(lambda x: x.split(" ")[-1]))
codes = codes[['자치구','읍면동명','adm_cd']].rename(columns={"adm_cd":"읍면동코드"})
codes = codes.assign(읍면동명 = codes.읍면동명.apply(lambda x: x.replace("·",".")))

In [7]:
data = pd.merge(data,codes, on='읍면동명')

In [8]:
data = data[(data.읍면동명!="신사동")|(data.자치구_x==data.자치구_y)]

In [9]:
data = data.drop(["자치구_x","자치구_y"],axis=1)

## 3. 기타 데이터와 병합

동별 기타 정보가 들어있는 데이터와 병합해준다.

In [10]:
pop = pd.read_csv("data/pop.csv")
pop = pop[pop.year==2016].drop(["Unnamed: 0", "year"],axis=1)
pop = pop.assign(읍면동명 = pop.읍면동명.apply(lambda x: x.replace("·",".")))

In [11]:
pop = pop.drop(["선거인수","새누리당","더불어민주당","국민의당"],axis=1)

In [12]:
신사 = pd.merge(data[data.읍면동명=='신사동'], pop[pop.읍면동명=='신사동'])
신사 = 신사[신사.선거구.apply(lambda x: x[:-1]) == 신사.자치구]

In [13]:
rest = pd.merge(data[data.읍면동명!='신사동'], pop[pop.읍면동명!='신사동'].drop("자치구",axis=1), on='읍면동명')

In [14]:
data = pd.concat([신사,rest],sort=False)

## 4. 선거구별 정리

In [15]:
elected = data.groupby("선거구")[['새누리당','더불어민주당','국민의당']].sum().apply(lambda x: x.idxmax(),1).reset_index()
elected.columns = ['선거구','당선']

In [16]:
data = pd.merge(data,elected).drop('자치구',axis=1)

## 5. 지리 데이터와 병합

In [17]:
import geopandas as gpd
import json
geodf = gpd.read_file("data/행정동2016.geojson")

In [18]:
geodf = geodf.astype({"adm_cd":"int"}).rename(columns={"adm_cd":"읍면동코드"})
geodf=geodf[['읍면동코드','geometry']]

In [19]:
data = pd.merge(geodf,data).rename(columns={"소득(동별)":"소득"})

In [22]:
type(data)

geopandas.geodataframe.GeoDataFrame

In [192]:
with open("data/election.json", "w") as jsonfile:
    json.dump(json.loads(data.to_json()), jsonfile)

In [194]:
data

,읍면동코드,geometry,읍면동명,선거인수,투표수,새누리당,더불어민주당,국민의당,선거구,독거노인수,...,기초수급자인원수,보육시설,금융기관,사업체 평균연령,장애등급별 장애인현황,인구,면적,인구밀도,소득,당선
0,1101053,"POLYGON ((126.97689 37.57565, 126.97703 37.569...",사직동,8199,5280,2486,2383,263,종로구,383.0,...,102,818.0,21.0,10.417,315.0,10013,1.23,8141,1.669880e+06,더불어민주당
1,1101054,"POLYGON ((126.98269 37.59507, 126.98337 37.594...",삼청동,2442,1518,680,707,85,종로구,156.0,...,50,32.0,5.0,8.000,126.0,3118,1.49,2093,5.186958e+05,더불어민주당
2,1101055,"POLYGON ((126.97585 37.59656, 126.97359 37.593...",부암동,8449,5396,2175,2806,252,종로구,319.0,...,140,145.0,2.0,10.250,334.0,11044,2.27,4865,1.811906e+06,더불어민주당
3,1101056,"POLYGON ((126.98014 37.62835, 126.98013 37.628...",평창동,15372,9780,4619,4571,387,종로구,615.0,...,124,195.0,5.0,10.250,573.0,19457,8.87,2194,3.241634e+06,더불어민주당
4,1101057,"POLYGON ((126.96067 37.58080, 126.96281 37.579...",무악동,6431,4418,1840,2241,248,종로구,205.0,...,129,250.0,1.0,11.999,300.0,8516,0.36,23656,1.505475e+06,더불어민주당
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,1125074,"POLYGON ((127.14857 37.54578, 127.14871 37.545...",길동,41751,23096,9346,9524,3447,강동구갑,1162.0,...,895,1282.0,9.0,8.250,1981.0,51434,2.17,23702,8.057254e+05,더불어민주당
420,1124054,"POLYGON ((127.14897 37.49628, 127.15032 37.495...",거여2동,17544,10595,4158,4721,1561,송파구병,637.0,...,541,650.0,3.0,9.667,1000.0,19609,0.53,36998,5.480795e+05,더불어민주당
421,1124055,"POLYGON ((127.16010 37.49716, 127.16005 37.494...",마천1동,18270,10130,4042,4367,1617,송파구병,816.0,...,1021,523.0,1.0,9.250,1258.0,22912,0.58,39503,6.698035e+05,더불어민주당
422,1124081,"POLYGON ((127.13667 37.48915, 127.13535 37.481...",장지동,27317,16604,6081,7704,2625,송파구병,663.0,...,916,1053.0,3.0,8.250,1477.0,35834,1.37,26156,1.096770e+06,더불어민주당
